In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
df_train = pd.read_csv('../input/nlp-getting-started/train.csv')
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
#Removing columns not relevant to our prediction
df_train.drop(['keyword','location'],axis=1,inplace=True)
df_test.drop(['keyword','location'],axis=1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
print(df_train.shape)
print(df_test.shape)

# Exploratory Data analysis

In [ ]:
g=sns.countplot(x='target',data=df_train)

Our training data is not skewed. We have almost equal distributions for each target value

<h2>Number of characters in tweets</h2>

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
len_1 = df_train[df_train['target']==1]['text'].str.len()
ax1.hist(len_1,color='red')
len_2 = df_train[df_train['target']==0]['text'].str.len()
ax2.hist(len_1,color='green')
ax1.set_title('Disaster tweet analysis')
ax2.set_title('Non-disaster tweet analysis')

The distribution among the both are very similar. The most common tweets comprise of about 120-140 characters.

<h2>Number of words in a tweet</h2>

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
sent_len1 = df_train[df_train['target']==1]['text'].str.split().map(lambda x:len(x))
ax1.hist(sent_len1,color='red')
sent_len0 = df_train[df_train['target']==0]['text'].str.split().map(lambda x:len(x))
ax2.hist(sent_len0,color='green')
ax1.set_title('Disaster tweets')
ax2.set_title('Non disaster tweets')

<h2>Average Word length</h2>

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
avg_1 = df_train[df_train['target']==1]['text'].str.split().apply(lambda x:[len(i) for i in x])
avg_0 = df_train[df_train['target']==0]['text'].str.split().apply(lambda x:[len(i) for i in x])
sns.distplot(avg_1.map(lambda x:np.mean(x)),ax=ax1,color='red')
sns.distplot(avg_0.map(lambda x:np.mean(x)),ax=ax2,color='green')
ax1.set_title('Disaster tweets')
ax2.set_title('Non disaster tweets')

<h2>Stopword analysis in tweets</h2>

In [ ]:
def make_corpus(t):
    res = []
    for x in df_train[df_train['target']==t]['text'].str.split():
        for i in x:
            res.append(i)
    return res

In [ ]:
f = make_corpus(0)

In [ ]:
# d1 = defaultdict(int)
# for word in f:
#     if word in stop:
#         d1[word]+=1

# print(d1)

d1 = Counter(f)

In [ ]:
top=sorted(d1.items(), key=lambda x:x[1],reverse=True)[:10] 
print(top)

In [ ]:
#Unzip the dictionary
x,y = zip(*top)
plt.bar(x,y)

In [ ]:
#Similar process for the target 1
corpus=make_corpus(1)

d2=defaultdict(int)
for word in corpus:
    if word in stop:
        d2[word]+=1

top=sorted(d2.items(), key=lambda x:x[1],reverse=True)[:10] 
    
x,y=zip(*top)
plt.bar(x,y)

<h2>Checking punctuation</h2>

In [ ]:
c1 = make_corpus(0)

d1 = defaultdict(int)
import string
s1 = string.punctuation
print(s1)

In [ ]:
plt.figure(figsize=(12,6))
for i in c1:
    if i in s1:
        d1[i]+=1

x,y = zip(*d1.items())
plt.bar(x,y)

In [ ]:
plt.figure(figsize=(12,6))
c2 = make_corpus(1)
d2 = defaultdict(int)
for i in c2:
    if i in s1:
        d2[i]+=1
x,y = zip(*d2.items())
plt.bar(x,y)

<h2>Common words</h2>

In [ ]:
plt.figure(figsize=(6,6))
f1 = make_corpus(1)
c1 = Counter(f1)
frequent1 = c1.most_common()
x = []
y = []
for a,b in frequent1[:10]:
    x.append(a)
    y.append(b)
    
sns.barplot(x,y)

In [ ]:
plt.figure(figsize=(6,6))
f0 = make_corpus(0)
c0 = Counter(f1)
frequent0 = c1.most_common()
x = []
y = []
for a,b in frequent0[:10]:
    x.append(a)
    y.append(b)
    
sns.barplot(x,y)

# Data cleaning

In [ ]:
t_len = len(df_train)
test_len  = len(df_test)
final = pd.concat([df_train,df_test],axis=0)

# Maximum sentence length

In [ ]:
final.head()

In [ ]:
p1 = final['text'].str.split().apply(lambda x:len(x))
max(p1)

In [ ]:
print(final.shape)
print(t_len)
print(test_len)

<h3>Remove URLS</h3>

In [ ]:
def url_removal(a):
    b = re.compile(r'https?://\S+|www\.\S+')
    return b.sub(r'',a)

a = 'hello this is www.kaggle.com'
b = 'follow me on https://www.gmail.com'

print(url_removal(a))
print(url_removal(b))

In [ ]:
final['text'] = final['text'].apply(lambda x: url_removal(x))

<h3>Removing HTML tags</h3>

In [ ]:
def html_removal(a):
    b = re.compile(r'<.*?>')
    return b.sub(r'',a)

a = '<h1>This is</h1>'
b = 'tagging<b>'

print(html_removal(a))
print(html_removal(b))

<h3>Removing Punctuation</h3>

In [ ]:
def punctuation_remove(a):
    b = str.maketrans('','',string.punctuation)
    return a.translate(b)

a = 'Hello, this is me!!!'
b = 'Excuse me: What are you doing here?'
print(punctuation_remove(a))
print(punctuation_remove(b))

In [ ]:
final['text'] = final['text'].apply(lambda x: punctuation_remove(x))

<h3>Spell checking</h3>

In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
s = SpellChecker()
def corrections(text):
    corrected = []
    wrong = s.unknown(text.split())
    for word in text.split():
        if word in wrong:
            corrected.append(s.correction(word))
        else:
            corrected.append(word)
    return ' '.join(corrected)

a = 'pleese corect me and fix my mistke'
print(corrections(a))

In [ ]:
#final['text']=final['text'].apply(lambda x : corrections(x))

# ML modeling

In [ ]:
#Function to get roc-auc score for validation
def roc_auc(predictions,target):
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [ ]:
#Making a backup copy
m1 = final.copy()

In [ ]:
train = final[:t_len]
test = final[t_len:]

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
test.drop(['target'],axis=1,inplace=True)

In [ ]:
train_y = train['target']
train.drop(['target'],axis=1,inplace=True)

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(train,train_y,test_size=0.1)

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

<h2>Basic models</h2>

Our very first model will focus on simple TF-IDF (Term frequency - inverse document frequency) followed by simple classification models.

In [ ]:
tfv = TfidfVectorizer()
tfv.fit(train['text'])
x_train_tfv = tfv.transform(x_train['text'])
x_valid_tfv = tfv.transform(x_valid['text'])

In [ ]:
print(x_train_tfv.shape)
print(x_valid_tfv.shape)

In [ ]:
print(x_train_tfv)

In [ ]:
len(tfv.get_feature_names())

In [ ]:
clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfv, y_train)
predictions = clf.predict(x_valid_tfv)

print ("ROC score: %0.3f " % roc_auc(y_valid, predictions))

In [ ]:
ctv = CountVectorizer()
ctv.fit(train['text'])
x_train_ctv = ctv.transform(x_train['text'])
x_valid_ctv = ctv.transform(x_valid['text'])

In [ ]:
clf = LogisticRegression(C=1.0)
clf.fit(x_train_ctv,y_train)
predictions = clf.predict(x_valid_ctv)

print ("ROC score: %0.3f " % roc_auc(y_valid, predictions))

# GloVe vectorization

In [ ]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [ ]:
corpus=create_corpus(m1)

In [ ]:
embeds = {}
with open('../input/glove6b100dtxt/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],'float32')
        embeds[word]=vectors
f.close()

In [ ]:
token = Tokenizer()
token.fit_on_texts(corpus)
sequence = token.texts_to_sequences(corpus)

In [ ]:
print('Number of unique words',len(token.word_index))

In [ ]:
max_len = 50
tweet_pad = pad_sequences(sequence,maxlen=max_len,truncating='post',padding='post')

In [ ]:
word_index = token.word_index
num_w = len(word_index)+1
embedding_matrix = np.zeros((num_w,100))

for word,i in tqdm(word_index.items()):
    if i>num_w:
        continue
    vec = embeds.get(word)
    if vec is not None:
        embedding_matrix[i] = vec
    

In [ ]:
embedding_matrix.shape

In [ ]:
model = Sequential()
embedding=Embedding(num_w,100,embeddings_initializer=Constant(embedding_matrix),input_length=max_len,trainable=False)
model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
optimzer=Adam(learning_rate=1e-5)
model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train = tweet_pad[:t_len]
test = tweet_pad[t_len:]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(train,train_y,test_size=0.1)

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, verbose=0, mode='auto',min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
history=model.fit(x_train,y_train,batch_size=4,epochs=20,validation_data=(x_val,y_val),verbose=1,callbacks=[reduce_lr])

In [ ]:
sample_sub=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
y_pre=model.predict(test)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('submission.csv',index=False)

In [ ]:
model = Sequential()
embedding=Embedding(num_w,100,embeddings_initializer=Constant(embedding_matrix),input_length=max_len,trainable=False)
model.add(embedding)
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
optimzer=Adam(learning_rate=1e-5)
model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [ ]:
history=model.fit(x_train,y_train,batch_size=4,epochs=20,validation_data=(x_val,y_val),verbose=1,callbacks=[reduce_lr])

In [ ]:
y_pre=model.predict(test)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('submissionbi.csv',index=False)